In [1]:
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import csv
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()

import ast
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from transformers import AutoModel, AutoTokenizer
from collections import defaultdict
import json

"""
2 corpora - before and after. split().-> bag of words - remove their names to @user for all of them within the content of the tweet not the person who tweeted. Pull the pre-trained bert vectors - hugging face - interate through the list - get the vectors of each words, dont remove duplicates (if dont find vectors for a word - have an output for those words - see if theres any spelling). we can also lemantize them, try to get hte vectors for those lemna, if still no vector - output, keep them somewhere. 
Dictionary: {Day: {tweet_id: {word}: []}, {word []}, etc.}

Once we have the list of words of dehumanizing language, count/number of words in your corpus: (e.g.: 2% were dehumanizing before, 7% after, frequency for each day and have a time-series analysis)
"""

/var/folders/wl/yn_f16552qvcz999s7pwv5sm0000gn/T/ipykernel_33480/319895550.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-11-18 22:17:21.526180: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


'\n2 corpora - before and after. split().-> bag of words - remove their names to @user for all of them within the content of the tweet not the person who tweeted. Pull the pre-trained bert vectors - hugging face - interate through the list - get the vectors of each words, dont remove duplicates (if dont find vectors for a word - have an output for those words - see if theres any spelling). we can also lemantize them, try to get hte vectors for those lemna, if still no vector - output, keep them somewhere. \nDictionary: {Day: {tweet_id: {word}: []}, {word []}, etc.}\n\nOnce we have the list of words of dehumanizing language, count/number of words in your corpus: (e.g.: 2% were dehumanizing before, 7% after, frequency for each day and have a time-series analysis)\n'

In [2]:
def remove_stop_words(text: string):
    return ' '.join([item for item in text.split() if item not in stopwords.words('english')])

def remove_punctuations(text: string):
    return "".join([char for char in text if char not in string.punctuation])

def delete_usernames(text: str) -> str:
    pattern = r'@\w+'
    return re.sub(pattern, '@user', text)

def remove_non_words(text: string):
    return re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(text))


# Function to split "After Corpus" into individual sentences and create a new DataFrame
def split_sentences(df, status: str):
    abbreviations = {
        "U.S.": "USA",
        "U.K.": "UK",
        "e.g.": "for example,",
        "i.e.": "such as,",
        "U.N.": "UN",
        "Gov.": "Governor",
        "etc.": "..."
    }

    # Initialize an empty list to store the rows of the new DataFrame
    rows = []

    # Iterate through each row in the original DataFrame
    for _, row in df.iterrows():
        # Extract the relevant data
        subject_id = row['Subject_ID']
        tweet_id = row['Tweet_ID']
        corpus = row[f'{status}_Corpus']

        # Replace abbreviations with placeholders
        for abbr, placeholder in abbreviations.items():
            corpus = corpus.replace(abbr, placeholder)
        
        # Split the corpus into sentences
        sentences = re.split(r'(?<=[.!?])\s+', corpus)

        # Add each sentence as a new row in the list, keeping track of the tweet ID and subject ID
        for sentence in sentences:
            rows.append({'Subject_ID': subject_id, 'Tweet_ID': tweet_id, 'Sentence': sentence})

    # Create a new DataFrame from the list of rows
    new_df = pd.DataFrame(rows)
    return new_df

# def stem_text(text: string):
#     return [ps.stem(word) for word in text]

def get_segments_ids(df):
    segments_ids = []
    for idx, row in df.iterrows():
        # Create a list of repeated IDs for each sentence
        segment_id_list = [idx] * len(row['Indexed_Tokens'])
        segments_ids.append(segment_id_list)
    df["Segments_IDs"] = segments_ids
    return df["Segments_IDs"]

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
#Pre-process Text Data
clean_data_path = "Cleaned Data"
after = pd.read_csv(f"{clean_data_path}/After_NN_Cleaned.csv")
before = pd.read_csv(f"{clean_data_path}/Before_NN_Cleaned.csv")

In [5]:
# Apply the function to split sentences
after_sentences = split_sentences(after, "After")
before_sentences = split_sentences(before, "Before")

# Apply all cleaning functions to the After Corpus column
after_sentences["Sentence"] = (after_sentences["Sentence"]
                             .apply(delete_usernames)
                             .apply(remove_punctuations)
                             .apply(remove_stop_words)
                             .apply(remove_non_words))

before_sentences["Sentence"] = (before_sentences["Sentence"]
                             .apply(delete_usernames)
                             .apply(remove_punctuations)
                             .apply(remove_stop_words)
                             .apply(remove_non_words))

# Apply sentence markers
after_sentences["Marked_Sentence"] = ("[CLS] "+ after_sentences["Sentence"] + " [SEP]")
before_sentences["Marked_Sentence"] = ("[CLS] "+ before_sentences["Sentence"] + " [SEP]")

# Show the first few rows of the resulting DataFrame
after_sentences.head()

,Subject_ID,Tweet_ID,Sentence,Marked_Sentence
0,74,609,hey remember phoned despot bestie Netanyahu to...,[CLS] hey remember phoned despot bestie Netany...
1,74,609,ace job homeslice,[CLS] ace job homeslice [SEP]
2,74,609,good thing meddling caused serious consequence...,[CLS] good thing meddling caused serious conse...
3,74,705,hey remember Jared Kushner brought peace Middl...,[CLS] hey remember Jared Kushner brought peace...
4,74,705,neither,[CLS] neither [SEP]


In [6]:
# Tokenize our sentence with the BERT tokenizer.
after_sentences["Tokens"] = after_sentences["Marked_Sentence"].apply(tokenizer.tokenize)
before_sentences["Tokens"] = before_sentences["Marked_Sentence"].apply(tokenizer.tokenize)

#Getting Indexed Tokens using BERT tokenzier
after_sentences["Indexed_Tokens"] = after_sentences["Tokens"].apply(lambda x: [tokenizer.convert_tokens_to_ids(token) for token in x])
before_sentences["Indexed_Tokens"] = before_sentences["Tokens"].apply(lambda x: [tokenizer.convert_tokens_to_ids(token) for token in x])

#Getting Segment IDs based on the number of sentences, all tokens in the same sentence get the same ID.
after_sentences["Segments_IDs"] = get_segments_ids(after_sentences)
before_sentences["Segments_IDs"] = get_segments_ids(before_sentences)

before_sentences.head()

,Subject_ID,Tweet_ID,Sentence,Marked_Sentence,Tokens,Indexed_Tokens,Segments_IDs
0,74,353,holy fucking shit criticizing Israel antisemitic,[CLS] holy fucking shit criticizing Israel ant...,"[[CLS], holy, fucking, shit, criticizing, isra...","[101, 4151, 8239, 4485, 21289, 3956, 3424, 336...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,74,353,IS antisemitic putting holocaustdenying lunati...,[CLS] IS antisemitic putting holocaustdenying ...,"[[CLS], is, anti, ##se, ##mit, ##ic, putting, ...","[101, 2003, 3424, 3366, 22930, 2594, 5128, 115...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,74,353,sit fuck shut fuck spare us fucking fake outrage,[CLS] sit fuck shut fuck spare us fucking fake...,"[[CLS], sit, fuck, shut, fuck, spare, us, fuck...","[101, 4133, 6616, 3844, 6616, 8622, 2149, 8239...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
3,74,538,congratulations literally zero Republicans sto...,[CLS] congratulations literally zero Republica...,"[[CLS], congratulations, literally, zero, repu...","[101, 23156, 6719, 5717, 10643, 2768, 7939, 23...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
4,74,538,take victory lap spineless fucking cowards,[CLS] take victory lap spineless fucking cowar...,"[[CLS], take, victory, lap, spine, ##less, fuc...","[101, 2202, 3377, 5001, 8560, 3238, 8239, 1659...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]"


In [7]:
after_sentences.to_csv("Tokenized Data/After_Tokenized.csv", index=False)
before_sentences.to_csv("Tokenized Data/Before_Tokenized.csv", index=False)